# Assignment

## Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance? For this assignment, the task is to predict whether a given blight ticket will be paid on time.

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

#  Data Clean

In [46]:
import pandas as pd
import numpy as np

# load data
train=pd.read_csv('C:/Users/yusha/Downloads/train.csv',encoding = "ISO-8859-1")
test= pd.read_csv('C:/Users/yusha/Downloads/test.csv',encoding = "ISO-8859-1")
address =  pd.read_csv('C:/Users/yusha/Downloads/addresses.csv')
latlons = pd.read_csv('C:/Users/yusha/Downloads/latlons.csv')

In [47]:
print(train.shape)
train.head()   

(250306, 34)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [48]:
print(test.shape)
test.head()

(61001, 27)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [49]:
print(address.shape)
address.head()

(311307, 2)


,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [50]:
print(latlons.shape)
latlons.head()

(121769, 3)


,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [51]:
train['compliance'].value_counts()  

0.0    148283
1.0     11597
Name: compliance, dtype: int64

In [52]:
train.isnull().sum()  # check missing value

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
payment_amount                     0
b

In [53]:
test.isnull().sum()  #check missing value

ticket_id                         0
agency_name                       0
inspector_name                    0
violator_name                    28
violation_street_number           0
violation_street_name             0
violation_zip_code            36977
mailing_address_str_number     1014
mailing_address_str_name          3
city                              1
state                           331
zip_code                          3
non_us_str_code               61001
country                           0
ticket_issued_date                0
hearing_date                   2197
violation_code                    0
violation_description             0
disposition                       0
fine_amount                       0
admin_fee                         0
state_fee                         0
late_fee                          0
discount_amount                   0
clean_up_cost                     0
judgment_amount                   0
grafitti_status               58780
dtype: int64

In [192]:
address.isnull().sum()

ticket_id    0
address      0
dtype: int64

In [194]:
latlons.isnull().sum()

address    0
lat        7
lon        7
dtype: int64

In [54]:
train.describe()

,ticket_id,violation_street_number,violation_zip_code,mailing_address_str_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,compliance
count,250306.000000,2.503060e+05,0.0,2.467040e+05,250305.000000,250306.000000,250306.000000,250306.000000,250306.000000,250306.0,250306.000000,250306.000000,250306.000000,159880.000000
mean,152665.543099,1.064986e+04,NaN,9.149788e+03,374.423435,12.774764,6.387382,21.494506,0.125167,0.0,268.685356,48.898986,222.449058,0.072536
std,77189.882881,3.188733e+04,NaN,3.602034e+04,707.195807,9.607344,4.803672,56.464263,3.430178,0.0,626.915212,222.422425,606.394010,0.259374
min,18645.000000,0.000000e+00,NaN,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-7750.000000,0.000000
25%,86549.250000,4.739000e+03,NaN,5.440000e+02,200.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,152597.500000,1.024400e+04,NaN,2.456000e+03,250.000000,20.000000,10.000000,10.000000,0.000000,0.0,140.000000,0.000000,25.000000,0.000000
75%,219888.750000,1.576000e+04,NaN,1.292725e+04,250.000000,20.000000,10.000000,25.000000,0.000000,0.0,305.000000,0.000000,305.000000,0.000000
max,366178.000000,1.415411e+07,NaN,5.111345e+06,10000.000000,20.000000,10.000000,1000.000000,350.000000,0.0,11030.000000,11075.000000,11030.000000,1.000000


In [55]:
# remove non-useful varaibles

train_remove = ['balance_due',
     'collection_status',
     'compliance_detail',
     'payment_amount',
     'payment_date',
     'payment_status']



remove_all = ['agency_name','violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code', 'state',
                          'ticket_issued_date', 'hearing_date', 'admin_fee', 'grafitti_status','state_fee','clean_up_cost',]


train.drop(train_remove,axis=1,inplace=True)
train.drop(remove_all,axis=1,inplace=True)
test.drop(remove_all,axis=1,inplace=True)

In [56]:
print(train.shape)
print(test.shape)
train.head()

(250306, 8)
(61001, 7)


,ticket_id,violation_code,disposition,fine_amount,late_fee,discount_amount,judgment_amount,compliance
0,22056,9-1-36(a),Responsible by Default,250.0,25.0,0.0,305.0,0.0
1,27586,61-63.0600,Responsible by Determination,750.0,75.0,0.0,855.0,1.0
2,22062,9-1-36(a),Not responsible by Dismissal,250.0,0.0,0.0,0.0,NaN
3,22084,9-1-36(a),Not responsible by City Dismissal,250.0,0.0,0.0,0.0,NaN
4,22093,9-1-36(a),Not responsible by Dismissal,250.0,0.0,0.0,0.0,NaN


In [57]:
train.isnull().sum()  # recheck the missing value in train data

ticket_id              0
violation_code         0
disposition            0
fine_amount            1
late_fee               0
discount_amount        0
judgment_amount        0
compliance         90426
dtype: int64

In [58]:
test.isnull().sum()  # recheck the missing value test data

ticket_id          0
violation_code     0
disposition        0
fine_amount        0
late_fee           0
discount_amount    0
judgment_amount    0
dtype: int64

In [59]:
train_data = train[(train['compliance'] == 0) | (train['compliance'] == 1)]  # extract compliance data with only 0 or 1 
train_data.shape   

(159880, 8)

In [60]:
print(train_data.violation_code.nunique())  # total 189 validation code
train_data.violation_code.value_counts()   # check the violation code

189


9-1-36(a)               64414
9-1-81(a)               23145
22-2-88                 19073
9-1-104                 16927
22-2-88(b)               4879
22-2-45                  4200
9-1-105                  3619
9-1-110(a)               3147
9-1-43(a) - (Dwellin     3043
9-1-103(C)               2641
22-2-22                  2612
22-2-43                  2054
19450901                 1620
22-2-17                  1414
22-2-61                   706
22-2-83(a)(b)(c)          494
9-1-82(d) - (Dwellin      457
61-81.0100/32.0066        439
9-1-43(a) - (Structu      399
22-2-83                   347
22-2-88(a)                235
9-1-206                   235
61-5-21                   202
9-1-209                   200
22-2-21(b)                185
9-1-101                   150
9-1-107                   133
9-1-43(a) - (Stories      132
9-1-111                   130
9-1-201(a)                128
                        ...  
20160901                    1
61-118.0100/32.0066         1
19850901  

In [61]:
vio_code_freq10 = train_data.violation_code.value_counts().index[0:10]    # transfer validation code to categorial data
vio_code_freq10

Index(['9-1-36(a)', '9-1-81(a)', '22-2-88', '9-1-104', '22-2-88(b)', '22-2-45',
       '9-1-105', '9-1-110(a)', '9-1-43(a) - (Dwellin', '9-1-103(C)'],
      dtype='object')

In [62]:
train_data['violation_code_freq10'] = [list(vio_code_freq10).index(c) 
                                       if c in vio_code_freq10 else -1 for c in train_data.violation_code ]
train_data['violation_code_freq10'].head()

C:\Users\yusha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    0
1   -1
5    0
6   -1
7   -1
Name: violation_code_freq10, dtype: int64

In [63]:
train_data['violation_code_freq10'].value_counts()   # change validation cade to 10 categorial data
train_data.drop('violation_code', axis=1, inplace=True)  # drop the original violation code

# change validation cade to 10 categorial data in test set
test['violation_code_freq10'] = [list(vio_code_freq10).index(c) if c in vio_code_freq10 else -1 for c in test.violation_code ]
test.drop('violation_code', axis=1, inplace=True)   

In [64]:
train_data.head()

,ticket_id,disposition,fine_amount,late_fee,discount_amount,judgment_amount,compliance,violation_code_freq10
0,22056,Responsible by Default,250.0,25.0,0.0,305.0,0.0,0
1,27586,Responsible by Determination,750.0,75.0,0.0,855.0,1.0,-1
5,22046,Responsible by Default,250.0,25.0,0.0,305.0,0.0,0
6,18738,Responsible by Default,750.0,75.0,0.0,855.0,0.0,-1
7,18735,Responsible by Default,100.0,10.0,0.0,140.0,0.0,-1


In [66]:
#change the categorial variables to dummy varialbes
dummy_columns = [ 'disposition', 'violation_code_freq10']
df_train = pd.get_dummies(train_data, columns=dummy_columns)
df_test = pd.get_dummies(test, columns=dummy_columns)

In [69]:
print(df_train.shape)
print(df_test.shape)

(159880, 21)
(61001, 24)


In [70]:
# merge latlons and addresses with data
df_train= pd.merge(df_train , pd.merge(address, latlons, on='address'), on='ticket_id')
df_test = pd.merge(df_test, pd.merge(address, latlons, on='address'), on='ticket_id')
df_train.drop('address',axis=1,inplace=True)
df_test.drop('address',axis=1,inplace=True)

In [74]:
print(df_train.shape)
print(df_test.shape)
print(df_train.columns.to_list())
print(df_test.columns.to_list())  
# we could see the number of features are different in train and test, so we need to remove columns that are not common

(159880, 23)
(61001, 26)
['ticket_id', 'fine_amount', 'late_fee', 'discount_amount', 'judgment_amount', 'compliance', 'disposition_Responsible (Fine Waived) by Deter', 'disposition_Responsible by Admission', 'disposition_Responsible by Default', 'disposition_Responsible by Determination', 'violation_code_freq10_-1', 'violation_code_freq10_0', 'violation_code_freq10_1', 'violation_code_freq10_2', 'violation_code_freq10_3', 'violation_code_freq10_4', 'violation_code_freq10_5', 'violation_code_freq10_6', 'violation_code_freq10_7', 'violation_code_freq10_8', 'violation_code_freq10_9', 'lat', 'lon']
['ticket_id', 'fine_amount', 'late_fee', 'discount_amount', 'judgment_amount', 'disposition_Responsible (Fine Waived) by Admis', 'disposition_Responsible (Fine Waived) by Deter', 'disposition_Responsible - Compl/Adj by Default', 'disposition_Responsible - Compl/Adj by Determi', 'disposition_Responsible by Admission', 'disposition_Responsible by Default', 'disposition_Responsible by Determination

In [92]:
# remove the columns that are not in train dataset
train_features = df_train.columns.drop('compliance')
X_train = df_train[train_features]
y_train = df_train.compliance
df_test=df_test[train_features]

In [93]:
train_features

Index(['ticket_id', 'fine_amount', 'late_fee', 'discount_amount',
       'judgment_amount', 'disposition_Responsible (Fine Waived) by Deter',
       'disposition_Responsible by Admission',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination', 'violation_code_freq10_-1',
       'violation_code_freq10_0', 'violation_code_freq10_1',
       'violation_code_freq10_2', 'violation_code_freq10_3',
       'violation_code_freq10_4', 'violation_code_freq10_5',
       'violation_code_freq10_6', 'violation_code_freq10_7',
       'violation_code_freq10_8', 'violation_code_freq10_9', 'lat', 'lon'],
      dtype='object')

In [95]:
X_train.drop('ticket_id',axis=1,inplace=True)
df_test.drop('ticket_id',axis=1,inplace=True)

In [100]:
X_train.head()

,fine_amount,late_fee,discount_amount,judgment_amount,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,violation_code_freq10_-1,violation_code_freq10_0,...,violation_code_freq10_2,violation_code_freq10_3,violation_code_freq10_4,violation_code_freq10_5,violation_code_freq10_6,violation_code_freq10_7,violation_code_freq10_8,violation_code_freq10_9,lat,lon
0,250.0,25.0,0.0,305.0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,42.390729,-83.124268
1,750.0,75.0,0.0,855.0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,42.326937,-83.135118
2,250.0,25.0,0.0,305.0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,42.145257,-83.208233
3,750.0,75.0,0.0,855.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,42.433466,-83.023493
4,100.0,10.0,0.0,140.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,42.388641,-83.037858


In [82]:
df_test.head()

,ticket_id,fine_amount,late_fee,discount_amount,judgment_amount,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,violation_code_freq10_-1,...,violation_code_freq10_2,violation_code_freq10_3,violation_code_freq10_4,violation_code_freq10_5,violation_code_freq10_6,violation_code_freq10_7,violation_code_freq10_8,violation_code_freq10_9,lat,lon
0,284932,200.0,20.0,0.0,250.0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,42.407581,-82.986642
1,285362,1000.0,100.0,0.0,1130.0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,42.426239,-83.238259
2,285361,100.0,10.0,0.0,140.0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,42.426239,-83.238259
3,285338,200.0,20.0,0.0,250.0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,42.309661,-83.122426
4,285346,100.0,10.0,0.0,140.0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,42.308830,-83.121116


In [117]:
#fill the missing value
X_train['lat']=X_train['lat'].fillna(method='pad')
X_train['lon']=X_train['lon'].fillna(method='pad')
df_test['lat']=df_test['lat'].fillna(method='pad')
df_test['lon']=df_test['lon'].fillna(method='pad')

C:\Users\yusha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\yusha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Build Models

In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, precision_score,recall_score, accuracy_score, precision_recall_curve
from sklearn.model_selection import GridSearchCV

In [123]:
#logistic model
grid_values = {'C': [0.01,0.05,0.1,1]}
clf_lg = LogisticRegression(solver = 'liblinear',random_state = 0)
lg = GridSearchCV(clf_lg, param_grid = grid_values,cv=5, scoring = 'roc_auc')
lg.fit(X_train, y_train)
print(lg.best_params_)
print(lg.best_score_)

{'C': 1}
0.7881786132449697


In [125]:
#random forests
rf = RandomForestClassifier(random_state = 0)
grid_values = {'n_estimators': [100,200,300,500], 'max_depth': [2,4,6,8]}
clf_rf = GridSearchCV(rf, param_grid=grid_values, cv = 5, scoring = "roc_auc")
clf_rf.fit(X_train, y_train)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

{'max_depth': 8, 'n_estimators': 500}
0.8033679452078044


In [127]:
#GradientBoosting Classifer
grid_values = {'learning_rate': [0.01, 0.1, 1]}
clf_gb= GradientBoostingClassifier(random_state = 0)
gb= GridSearchCV(clf_gb, param_grid = grid_values,cv=5,scoring = 'roc_auc')
gb.fit(X_train, y_train)
print(gb.best_params_)
print(gb.best_score_)

{'learning_rate': 0.1}
0.8025975310796973


In [131]:
#MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(df_test)
clf_mlp = MLPClassifier(hidden_layer_sizes = [500, 50], alpha = 0.00,
                    random_state = 0,
                    shuffle = False,
                    learning_rate_init = 0.005,
                    batch_size = 1000,
                    learning_rate = 'invscaling',
                    momentum = 0.1,
                    power_t = 0.001,
                    verbose = False,
                    max_iter = 400,
                    tol = 0.0001,
                    solver='adam')
clf_mlp.fit(X_train_scaled, y_train)

MLPClassifier(activation='relu', alpha=0.0, batch_size=1000, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=[500, 50], learning_rate='invscaling',
              learning_rate_init=0.005, max_iter=400, momentum=0.1,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.001,
              random_state=0, shuffle=False, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [133]:
print(clf_mlp.score(X_train_scaled, y_train))

0.9402176632474356


MLPClassifer model achieved highest score. Since we don't have target value in the test dataset. So I only predict the probability of each corresponding ticket

In [141]:
test_proba = clf_mlp.predict_proba(X_test_scaled)[:,1]
test['compliance'] = test_proba

In [152]:
test.head()

,disposition,fine_amount,late_fee,discount_amount,judgment_amount,violation_code_freq10,compliance
ticket_id,,,,,,,
284932,Responsible by Default,200.0,20.0,0.0,250.0,-1,0.041196
285362,Responsible by Default,1000.0,100.0,0.0,1130.0,4,0.008345
285361,Responsible by Default,100.0,10.0,0.0,140.0,-1,0.048511
285338,Responsible by Default,200.0,20.0,0.0,250.0,4,0.042039
285346,Responsible by Default,100.0,10.0,0.0,140.0,5,0.082315
